# Easy

In [1]:
import subprocess
hostname = "192.168.29.1"
speeds = []
for size in [65500,64000,60000,6550]:
    for i in range(10):
        response = subprocess.getoutput(f"ping {hostname} -n 1 -w 500 -l {size}")
        try:
            time_string = [text for text in response.split(" ") if 'time=' in text][0].split("=")[1]
            speeds.append(size*1000/(int(time_string[:-2])*1024*1024))
        except:
            pass
print(round(max(speeds),2), "Megabytes/sec")

10.41 Megabytes/sec


# Complex

In [2]:
import subprocess
import time
from multiprocessing import Pool
import concurrent.futures

In [3]:
def n_concurrent_ping_time(n, workers, method = 'multiprocessing', hostname = "192.168.29.1", size = 65500):
    if n == 1:
        response = subprocess.getoutput(f"ping {hostname} -n 1 -w 500 -l {size}")
        try:
            time_string = [text for text in response.split(" ") if 'time=' in text][0].split("=")[1]
            return int(time_string[:-2])/1000
        except:
            return -1
    
    input_list = n*[f"ping {hostname} -n 1 -w 500 -l {size}"]
    
    if method == 'multiprocessing':
        with Pool(workers) as p:
            results = p.map(len, input_list)
            t_start = time.perf_counter()
            results = p.map(subprocess.getoutput, input_list)
            t_stop = time.perf_counter()
    elif method == 'threads':
        with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as e:
            results = list(e.map(len, input_list))
            t_start = time.perf_counter()
            results = list(e.map(subprocess.getoutput, input_list))
            t_stop = time.perf_counter()
    else:
        raise "Unknown method"
        
    time_taken = t_stop - t_start
    for response in results:
        try:
            response = subprocess.getoutput(f"ping {hostname} -n 1 -w 500 -l {size}")
            time_string = [text for text in response.split(" ") if 'time=' in text][0].split("=")[1]
            ping_time = int(time_string[:-2])
        except:
            return -1
    return time_taken

In [4]:
speeds = []
size = 65500
for n in [1,2,3,4,8,10]:
    for workers in [1,2,3,4]:
        for method in ['multiprocessing', 'threads']:
            #print(n,workers,method)
            total_time = n_concurrent_ping_time(n,workers,method,"192.168.29.1",size)
            #print(round((n*size)/(total_time*1024*1024),2), "Megabytes/sec")
            speeds.append(round((n*size)/(total_time*1024*1024),2))
            time.sleep(0.2)
print(max(speeds), "Megabytes/sec")

10.41 Megabytes/sec
